In [1]:
from BrattEssay import load_bratt_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings

settings = Settings()

#essays = load_bratt_essays(settings.data_directory + "SkinCancer/EBA1415_Merged/")
folder = settings.data_directory + "CoralBleaching/BrattData/EBA1415_Merged/"
essays = load_bratt_essays(folder)

wd_sent_freq = defaultdict(int)
all_codes = set()
#Stores all words for the spelling corrector
words = []
all_sentences = []
sentencesForCode = defaultdict(list)
for essay in essays:
    for sentence in essay.tagged_sentences:
        wdsInSent = set()
        codes4sentence = set()
        sent = []
        for w, tags in sentence:
            words.append(w)
            all_codes.update(tags)
            codes4sentence.update(tags)
            if w not in wdsInSent:
                wdsInSent.add(w)
                wd_sent_freq[w] += 1
            sent.append(w)
        all_sentences.append(sent)
        for code in codes4sentence:
            sentencesForCode[code].append(sentence)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
(1154, 'files found')
Skipping /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/BrattData/EBA1415_Merged/EBA1415_AEKD_4_CB_ES-05571.ann file as .txt file is no essay'
Skipping /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/BrattData/EBA1415_Merged/EBA1415_AEKD_4_CB_ES-05904.ann file as .txt file is no essay'
Skipping /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/BrattData/EBA1415_Merged/EBA1415_BGJD_1_CB_ES-05733.ann file as .txt file is no essay //'
Skipping /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/BrattData/EBA1415_Merged/EBA1415_ERSK_7_CB_ES-05798.ann file as .txt file is no essay //'
Skipping /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/BrattData/EBA1415_Merged/EBA1415_KYLS_5_CB_ES-05671.a

### Compute Stats over the Essays

In [2]:
wd_counts = []
sent_counts = []
concept_codes = []
cr_concept_codes = []
sent_multi_word_tags = {}
sent_codes = []
sent_cr_codes = []

num_sents = 0
un_wd_counts = []
vocab = set()
for e_ix, essay in enumerate(essays):
    wds = 0
    un_words = set()
    for i, sentence in enumerate(essay.tagged_sentences):
        num_sents += 1
        sent_tags = set()
        sent_cr_tags = set()
        for w, tags in sentence:
            un_words.add(w)
            vocab.add(w)
            wds += 1
            ccodes = [t for t in tags if t[0].isdigit()]
            if ccodes:
                sent_tags.update(ccodes)
                concept_codes.append(ccodes)
                if len(ccodes) > 1:
                    sent_multi_word_tags[(e_ix, i)] = [(w,[tag for tag in t if tag[0].isdigit()]) for w,t in sentence]
            cr_codes = [t for t in tags if t[0].isdigit() or t == "Causer" or t == "Result" or t == "explicit"]
            if cr_codes:
                cr_concept_codes.append(cr_codes)
                sent_cr_tags.update(cr_codes)
        if len(sent_tags) > 0:
            sent_codes.append(sent_tags)
            if len(sent_cr_tags) > 0:
                sent_cr_codes.append(sent_cr_tags)
                
    un_wd_counts.append(len(un_words))
    sent_counts.append(len(essay.tagged_sentences))
    wd_counts.append(wds)

In [3]:
print len(sent_multi_word_tags), num_sents
sent_multi_word_tags.items()[0]

7 10210


((138, 5),
 [('The', []),
  ('strength', ['1', '3']),
  ('of', ['1', '3']),
  ('the', ['3']),
  ('water', ['3']),
  ('and', []),
  ('wind', ['1']),
  ('get', []),
  ('algae', ['7']),
  ('out', ['7']),
  ('of', ['7']),
  ('control', ['7']),
  ('and', []),
  ('turns', ['50']),
  ('coral', ['50']),
  ('white', ['50']),
  ('.', [])])

In [4]:
len(sent_codes), len([tags for tags in sent_codes if len(tags) > 1])

(7094, 3138)

In [5]:
sent_multi_word_tags.items()[1]

((640, 0),
 [('Corals', ['50']),
  ('get', ['50']),
  ('there', ['50']),
  ('color', ['50']),
  ('by', ['50']),
  ('the', ['50']),
  ('different', ['50']),
  ('types', ['50']),
  ('of', ['50']),
  ('algae', ['50', '7']),
  ('that', ['7']),
  ('live', ['7']),
  ('with', ['7']),
  ('the', ['7']),
  ('coral', ['7']),
  ('.', [])])

### Essay Length Statistics

In [6]:
import numpy as np
print np.mean(wd_counts), np.median(wd_counts), np.min(wd_counts), np.max(wd_counts), np.std(wd_counts)
print np.mean(sent_counts), np.median(sent_counts), np.min(sent_counts), np.max(sent_counts), np.std(sent_counts)
#148.824468085 142.0 1 461 81.6952326181
#9.05141843972 8.0 1 31 5.16578158718

148.824468085 142.0 1 461 81.6952326181
9.05141843972 8.0 1 31 5.16578158718


### Vocabulary Size

In [7]:
print("Total Unique Words:" , len(vocab))

('Total Unique Words:', 4776)


In [8]:
np.min(un_wd_counts), np.max(un_wd_counts), np.mean(un_wd_counts), np.median(un_wd_counts), np.std(un_wd_counts)
#(1, 226, 83.875, 83.5, 37.813481392223068)

(1, 226, 83.875, 83.5, 37.813481392223068)

In [9]:
from IterableFP import flatten
unique = set(flatten(concept_codes))
unique

{'1', '11', '12', '13', '14', '2', '3', '4', '5', '50', '5b', '6', '7'}

### How Many Tagged Words Have Multiple Codes?

In [10]:
print len(concept_codes), sum(wd_counts), round(len(concept_codes) / float(sum(wd_counts)),2)
multiple = [tags for tags in concept_codes if len(tags) > 1]
print len(multiple), len(multiple) / float(len(concept_codes))
multiple

52084 167874 0.31
12 0.000230397050918


[['50', '5b'],
 ['50', '5b'],
 ['1', '3'],
 ['1', '3'],
 ['50', '7'],
 ['7', '14'],
 ['7', '14'],
 ['50', '7'],
 ['13', '4'],
 ['13', '4'],
 ['13', '4'],
 ['14', '50']]

In [11]:
unique_cr = set(flatten(cr_concept_codes))
unique_cr

{'1',
 '11',
 '12',
 '13',
 '14',
 '2',
 '3',
 '4',
 '5',
 '50',
 '5b',
 '6',
 '7',
 'Causer',
 'Result',
 'explicit'}

In [12]:
print len(cr_concept_codes), sum(wd_counts), round(len(cr_concept_codes) / float(sum(wd_counts)),2)
multiple = [tags for tags in cr_concept_codes if len(tags) > 1]
print len(multiple), len(multiple) / float(len(cr_concept_codes))

57535 167874 0.34
28007 0.486781958808


### What Proportion of Sentences With Codes Have Multiple Codes?

In [13]:
num_sents = float(num_sents)
print len(sent_codes) / num_sents
num_multiple_codes = len([tags for tags in sent_codes if len(tags) > 1])
num_multiple_codes / float(len(sent_codes))

0.694809010774


0.44234564420637157

In [14]:
len(sent_cr_codes) / num_sents
num_multiple_codes = len([tags for tags in sent_cr_codes if len(tags) > 1])
num_multiple_codes / float(len(sent_cr_codes))

0.5

### Conditional Probabilities of Codes

In [15]:
from collections import defaultdict
priors = defaultdict(float)
joints = defaultdict(float)

for sent in sent_codes:
    for a in sorted(sent):
        priors[a] += 1
        for b in sorted(sent):
            if b >= a:
                break
            joints[(b,a)] +=1

conditional = {}
for a, cnt in priors.items():
    for b in priors.keys():
        if a == b:
            continue
        """ p(A/B) = p(B/A)p(A) / p(B) """
        """ p(A/B) = p(B/A)p(A) """
        if a < b:
            joint = joints[(a,b)]
        else:
            joint = joints[(b,a)]
        conditional[(a,b)] = joint / priors[b]
    
lifts = {}
total = float(sum(joints.values()))
totalPrior = float(sum(priors.values()))
for (a,b),cnt in joints.items():
    joint = cnt / total
    pA = priors[a] / totalPrior
    pB = priors[b] / totalPrior
    lift = joint / (pA * pB)
    if lift:
        lifts[(a,b)] = lift

In [16]:
def get_num(a):
    s = ""
    for c in a:
        if c.isdigit():
            s += c
    return int(s)

consec_pmi = {}
for (a,b), lift in lifts.items():
    ia = get_num(a)
    ib = get_num(b)
    diff = abs(ia-ib)
    pmi = np.log(lift)
    if "Coral" in folder:
        if diff == 1 and b != "6" and b != "5b":
            consec_pmi[(a,b)] = pmi
        elif a =="5" and b =="5b":
            consec_pmi[(a,b)] = pmi
        elif a =="14" and b =="5b":
            consec_pmi[(a,b)] = pmi
        elif a =="14" and b =="6":
            consec_pmi[(a,b)] = pmi
        elif a =="50" and b =="7":
            consec_pmi[(a,b)] = pmi
    elif "Skin" in folder:
        if diff == 1:
            consec_pmi[(a,b)] = pmi
        elif a == "12" and b == "6":
            consec_pmi[(a,b)] = pmi
        elif a == "50" and b == "6":
            consec_pmi[(a,b)] = pmi        
            
for k,v in sorted(consec_pmi.items(), key = lambda (k,v): (min(int(k[0]),int(k[1].replace("b",""))))):
    a = k[0]
    b = k[1]
    if len(a) > len(b.replace("b","")):
        a,b = b,a
    print "&", str(a).ljust(5), "&", str(b).ljust(5), "&", str(round(v,2)).rjust(5), "\\\\"

& 1     & 2     &  2.23 \\
& 2     & 3     &  1.53 \\
& 3     & 4     &  1.73 \\
& 4     & 5     &  2.11 \\
& 5b    & 14    & -0.69 \\
& 5     & 5b    &  2.11 \\
& 6     & 7     &  2.24 \\
& 6     & 14    &  2.34 \\
& 7     & 50    &   1.2 \\
& 11    & 12    &  3.93 \\
& 12    & 13    &  3.58 \\
& 13    & 14    &  2.59 \\


In [17]:
np.mean(lifts.values())
#sorted(lifts.items(), key=lambda (tpl,p):-p)

3.8575045329169773

In [18]:
np.mean(map(lambda l: np.log(l), lifts.values()))

0.400765095332002

## Compute Relative Class Frequencies At Word and Sentence Levels (Class Imbalances)

In [19]:
wd_cc_tally = defaultdict(int)
sent_cc_tally = defaultdict(int)
wd_cnt = 0
sent_cnt = 0

for e_ix, essay in enumerate(essays):

    for i, sentence in enumerate(essay.tagged_sentences):
        sent_cnt += 1
        sent_tags = set()
        
        for w, tags in sentence:
            wd_cnt += 1
            # Concept Codes
            ccodes = [t for t in tags if t[0].isdigit()]
            if ccodes:
                for cc in ccodes:
                    wd_cc_tally[cc] += 1
                    sent_tags.add(cc)
            #Causal Codes
            #This is wrong
            #cr_codes = [t for t in tags if t[0].isdigit() or t == "Causer" or t == "Result" or t == "explicit"]
            #if cr_codes:
                 
        if len(sent_tags) > 0:
            for tag in sent_tags:
                sent_cc_tally[tag] +=1            

In [20]:
print(wd_cnt, sent_cnt)
#Code, Number of Words, Proportion
for code, wd_tally in sorted(wd_cc_tally.items(), key = lambda tpl: int(tpl[0].replace("b",""))):
    wd_pct = 100* (float(wd_tally) / wd_cnt)
    sent_pct = 100*(float(sent_cc_tally[code]) / sent_cnt)
    #print(code.ljust(2), str(tally).ljust(6), "{0:.4f}".format(float(tally)/wd_cnt))
    # Print Latex Output (Near End of chapter 2)
    print("{code} & {wd_pct:.2f} & {sent_pct:.2f} \\\\".format(code=code.ljust(3),wd_pct=wd_pct, sent_pct=sent_pct))

(167874, 10210)
1   & 3.32 & 12.69 \\
2   & 0.85 & 2.26 \\
3   & 5.36 & 14.61 \\
4   & 1.89 & 5.41 \\
5   & 1.45 & 6.25 \\
5b  & 1.45 & 2.70 \\
6   & 0.88 & 4.01 \\
7   & 3.03 & 9.17 \\
11  & 0.63 & 3.66 \\
12  & 0.51 & 1.22 \\
13  & 1.34 & 4.92 \\
14  & 1.47 & 3.34 \\
50  & 8.86 & 39.38 \\


### Print Counts for Metrics By Class Notebook (See Mongo Queries)

In [21]:
from pprint import pprint
print(wd_cnt)
print("")
pprint(dict(wd_cc_tally))

167874

{'1': 5581,
 '11': 1052,
 '12': 851,
 '13': 2244,
 '14': 2470,
 '2': 1426,
 '3': 8998,
 '4': 3165,
 '5': 2440,
 '50': 14873,
 '5b': 2434,
 '6': 1471,
 '7': 5091}


## Compute Relative Frequencies of Each Causal Relation (Word and Sentence Level)

In [22]:
wd_cr_tally = defaultdict(int)
sent_cr_tally = defaultdict(int)
wd_cr_cnt = 0
sent_cr_cnt = 0

for e_ix, essay in enumerate(essays):

    for i, sentence in enumerate(essay.tagged_sentences):
        sent_cr_cnt += 1
        sent_tags = set()
        sent_cr_tags = set()
        
        for w, tags in sentence:
            wd_cr_cnt += 1
            # Concept Codes
            cr_codes = list((t for t in tags if ( "->" in t) and not "factor" in t and not "Anaphor" in t and not "other" in t and not "rhetorical" in t))
            if cr_codes:
                for cr in cr_codes:
                    wd_cr_tally[cr] += 1
                    sent_tags.add(cr)           
        if len(sent_tags) > 0:
            for tag in sent_tags:
                sent_cr_tally[tag] +=1            

In [23]:
def cc_to_float(cc):
    if "b" in cc:
        cc = cc.replace("b","")
        cc += ".1"
    return float(cc)

def sort_key(tpl):
    cr,cnt = tpl
    cr = cr.replace("Causer:","")
    cr = cr.replace("Result:","")
    l,r = cr.split("->")
    l,r = cc_to_float(l), cc_to_float(r)
    return (l,r)

srtd_cnts = sorted(wd_cr_tally.items(), key = sort_key)

prev = ""

lines = []
for cr, wd_tally in srtd_cnts:
    l,r = cr.split("->")
    if l != prev:
        #print("\cmidrule{1-3}")
        prev = l
    wd_pct = 100 * (float(wd_tally) / wd_cr_cnt)
    sent_pct = 100 *(float(sent_cr_tally[cr]) / sent_cnt)
    # Print Latex Output (Near End of chapter 2)
    cr = cr.replace("->","\\textrightarrow ")
    cr = cr.replace("Causer:","").replace("Result:","")
    lines.append("{cr} & {wd_pct:.2f} & {sent_pct:.2f}".format(cr=cr.ljust(25),wd_pct=wd_pct, sent_pct=sent_pct))

In [24]:
len(lines)

86

In [25]:
half = int(len(lines)+1)//2
if len(lines) % 2 != 0:
    half += 1

for i in range(0,half):
    left = lines[i]
    right = ""
    if i + half  < (len(lines)):
        right = lines[i+half]
    print(left + " & & " + right + " \\\\")

1\textrightarrow 2        & 0.70 & 1.05 & & 5b\textrightarrow 14      & 0.01 & 0.02 \\
1\textrightarrow 3        & 1.75 & 2.60 & & 5b\textrightarrow 50      & 0.27 & 0.35 \\
1\textrightarrow 4        & 0.05 & 0.08 & & 6\textrightarrow 5        & 0.01 & 0.01 \\
1\textrightarrow 5        & 0.04 & 0.06 & & 6\textrightarrow 5b       & 0.00 & 0.01 \\
1\textrightarrow 6        & 0.00 & 0.01 & & 6\textrightarrow 7        & 1.17 & 1.52 \\
1\textrightarrow 7        & 0.05 & 0.07 & & 6\textrightarrow 14       & 0.41 & 0.55 \\
1\textrightarrow 11       & 0.01 & 0.01 & & 6\textrightarrow 50       & 0.48 & 0.73 \\
1\textrightarrow 13       & 0.02 & 0.02 & & 7\textrightarrow 1        & 0.01 & 0.01 \\
1\textrightarrow 14       & 0.02 & 0.03 & & 7\textrightarrow 4        & 0.01 & 0.01 \\
1\textrightarrow 50       & 2.57 & 4.26 & & 7\textrightarrow 5        & 0.01 & 0.01 \\
2\textrightarrow 1        & 0.01 & 0.01 & & 7\textrightarrow 5b       & 0.03 & 0.04 \\
2\textrightarrow 3        & 0.41 & 0.50 & &

## Compute Aggregate Frequences of Concept Codes (Word and Sentence Level)

In [100]:
l = list(range(10))
print(l)
l.pop?
print(l.pop())
print(l)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
9
[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [109]:
num_sents = 0
sents_with_codes = 0
# sents where one or more words has overlapping codes
sents_with_overlapping_codes = 0
# sents with >1 code
sents_with_multiple_codes = 0

num_words = 0
words_with_codes = 0
words_with_overlapping_codes = 0

seq_lens = defaultdict(list)
long_seqs = defaultdict(set)

for e_ix, essay in enumerate(essays):
    for i, sentence in enumerate(essay.tagged_sentences):    
        has_code_in_sent = False
        has_overlapping_codes_in_sent = False
        unique_ccodes = set()
        
        last_ccodes = set()
        for w, tags in sentence:
            # Concept Codes
            ccodes = set([t for t in tags if t[0].isdigit()])
            # update word stats
            if ccodes:
                unique_ccodes.update(ccodes)
                has_code_in_sent = True
                words_with_codes +=1
                
                if len(ccodes) > 1:
                    words_with_overlapping_codes +=1
                    has_overlapping_codes_in_sent = True
            num_words +=1
            # end update word stats
            
            # Update seq length stats
            for code in ccodes:
                if code in last_ccodes:
                    # get list
                    lst = seq_lens[code]
                    # remove last entry
                    last_item = lst.pop()
                    lst.append(last_item+1)
                    # store indexes for debugging
                    #if last_item > 15:
                    #    long_seqs[code].add((e_ix,i))
                else:
                    seq_lens[code].append(1)
            # reset last_ccodes
            last_ccodes = ccodes            
            # end update seq length stats
        # update sent stats
        if has_code_in_sent:
            sents_with_codes +=1
            if has_overlapping_codes_in_sent:
                sents_with_overlapping_codes +=1
            if len(unique_ccodes) > 1:
                sents_with_multiple_codes +=1
        num_sents +=1
        # end update sent       

In [75]:
num_words, words_with_codes, words_with_overlapping_codes

(167874, 52084, 12)

In [86]:
num_sents, sents_with_codes, sents_with_overlapping_codes, sents_with_multiple_codes

(10210, 7094, 7, 3138)

In [90]:
from __future__ import division

print("% words with codes: \t \t \t{pct:.3f}%".format(pct=100*words_with_codes/ num_words) )
print("% words with multiple codes: \t \t{pct:.4f}%".format(pct=100*words_with_overlapping_codes/ num_words) )
print("")
print("% sents with codes: \t \t \t{pct:.3f}%".format(pct=100*sents_with_codes/ num_sents) )
print("% sents with wds. with mult. codes: \t{pct:.3f}%".format(pct=100*sents_with_overlapping_codes/ num_sents) )
print("% sents with multiple codes: \t \t{pct:.3f}%".format(pct=100*sents_with_multiple_codes/ num_sents) )


% words with codes: 	 	 	31.026%
% words with multiple codes: 	 	0.0071%

% sents with codes: 	 	 	69.481%
% sents with wds. with mult. codes: 	0.069%
% sents with multiple codes: 	 	30.735%


### Sequence Lengths

In [127]:
all_lens = []
for code in sorted(seq_lens.keys()):
    print("{code} - {mean_len:.2f}".format(code=code.ljust(10), mean_len=np.mean(seq_lens[code])))
    all_lens.extend(seq_lens[code])
print
print("{code} - {mean_len:.2f}".format(code="All".ljust(10), mean_len=np.mean(all_lens)))

1          - 3.88
11         - 2.47
12         - 6.60
13         - 4.01
14         - 7.06
2          - 5.75
3          - 4.31
4          - 4.91
5          - 3.59
50         - 3.32
5b         - 6.26
6          - 3.45
7          - 4.40

All        - 4.00


## Compute Aggregate Frequencies of Causal Relations (Word and Sentence Level)

In [128]:
num_sents = 0
sents_with_causal = 0
sents_with_overlapping_causal = 0
sents_with_multiple_causal = 0

num_words = 0
words_with_causal = 0
words_with_overlapping_causal = 0

cr_sents = []

cr_seq_lens = defaultdict(list)

for e_ix, essay in enumerate(essays):

    for i, sentence in enumerate(essay.tagged_sentences):    
        has_cr_in_sent = False
        has_overlapping_cr_in_sent = False
        unique_causal = set()
        last_cr_codes = set()
        
        for w, tags in sentence:
            # Concept Codes
            cr_codes = set((t for t in tags if ( "->" in t) 
                             and not "factor" in t 
                             and not "Anaphor" in t 
                             and not "other" in t 
                             and not "rhetorical" in t))
            # update word stats
            if cr_codes:
                unique_causal.update(cr_codes)
                has_cr_in_sent = True
                words_with_causal +=1
                
                if len(cr_codes) > 1:
                    words_with_overlapping_causal +=1
                    has_overlapping_cr_in_sent = True
            num_words +=1
            # end update word stats
            
            # Update seq length stats
            for code in cr_codes:
                if code in last_cr_codes:
                    # get list
                    lst = cr_seq_lens[code]
                    # remove last entry
                    last_item = lst.pop()
                    lst.append(last_item+1)
                else:
                    cr_seq_lens[code].append(1)
            # reset last_ccodes
            last_cr_codes = cr_codes
                    
        # update sent stats
        if has_cr_in_sent:
            cr_sents.append(sentence)
            sents_with_causal +=1
            if has_overlapping_cr_in_sent:
                sents_with_overlapping_causal +=1
            if len(unique_causal) > 1:
                sents_with_multiple_causal +=1
        num_sents +=1
        # end update sent       

In [129]:
num_words, words_with_causal/float(num_words)

(167874, 0.21061629555499958)

In [130]:
num_sents, sents_with_causal/ float(num_sents)

(10210, 0.272771792360431)

In [131]:
num_words, words_with_causal, words_with_overlapping_causal, words_with_overlapping_causal/float(words_with_causal)

(167874, 35357, 4892, 0.1383601549905252)

In [132]:
num_sents, sents_with_causal, sents_with_overlapping_causal, sents_with_multiple_causal, sents_with_overlapping_causal/ float(sents_with_causal)

(10210, 2785, 621, 678, 0.2229802513464991)

In [133]:
from __future__ import division

print("% words with cr codes: \t \t \t {pct:.4f}%".format(pct=100*words_with_causal/ num_words) )
print("% words with multiple cr codes: \t {pct:.4f}%".format(pct=100*words_with_overlapping_causal/ num_words) )
print("")
print("% sents with cr codes: \t \t \t {pct:.4f}%".format(pct=100*sents_with_causal/ num_sents) )
print("% sents with multiple cr codes: \t {pct:.4f}%".format(pct=100*sents_with_multiple_causal/ num_sents) )


% words with cr codes: 	 	 	 21.0616%
% words with multiple cr codes: 	 2.9141%

% sents with cr codes: 	 	 	 27.2772%
% sents with multiple cr codes: 	 6.6405%


### Causal Relation Sequence Lengths

In [134]:
all_cr_lens = []
for code in sorted(cr_seq_lens.keys()):
    #print("{code} - {mean_len:.2f}".format(code=code.ljust(10), mean_len=np.mean(cr_seq_lens[code])))
    all_cr_lens.extend(cr_seq_lens[code])
print("{code} - {mean_len:.2f}".format(code="All".ljust(10), mean_len=np.mean(all_cr_lens)))

All        - 7.87
